In [1]:
KB = 1024.0
MB = 1024 * KB
GB = 1024 * MB

In [2]:
QLOG = 2.2 * MB
QCAM = QLOG

RLOG = 56 * MB
DCAM = ECAM = FCAM = 73 * MB

In [3]:
TRAINING = RLOG + DCAM + ECAM + FCAM  # TODO: toggle dcam
DASHCAM = QLOG + QCAM
print(f"Training={TRAINING/MB}MB Dashcam={DASHCAM/MB:.2f}MB")
print(f"Dashcam / Training (%) = {DASHCAM/TRAINING*100:.2f}%")

Training=275.0MB Dashcam=4.40MB
Dashcam / Training (%) = 1.60%


In [4]:
# What if we increased qcam size and had three of them?
QCAM = (8.43 * MB) * 3
DASHCAM = QLOG + QCAM
print(f"Training={TRAINING/MB}MB Dashcam={DASHCAM/MB:.2f}MB")
print(f"Dashcam / Training (%) = {DASHCAM/TRAINING*100:.2f}%")

Training=275.0MB Dashcam=27.49MB
Dashcam / Training (%) = 10.00%


In [5]:
# What if we increased qcam size to 12MB?
QCAM = (10 * MB) * 3
DASHCAM = QLOG + QCAM
print(f"Training={TRAINING/MB}MB Dashcam={DASHCAM/MB:.2f}MB")
print(f"Dashcam / Training (%) = {DASHCAM/TRAINING*100:.2f}%")

Training=275.0MB Dashcam=32.20MB
Dashcam / Training (%) = 11.71%


In [6]:
# What if each camera stream was 15MB?
QCAM = (15.9 * MB) * 3
DASHCAM = QLOG + QCAM
print(f"Training={TRAINING/MB}MB Dashcam={DASHCAM/MB:.2f}MB")
print(f"Dashcam / Training (%) = {DASHCAM/TRAINING*100:.2f}%")

Training=275.0MB Dashcam=49.90MB
Dashcam / Training (%) = 18.15%


In [7]:
# comma 3X with OP 0.9.7
RESERVED_BYTES = (4.5 + 3.1) * GB  # 4.5G staging, 3.1G openpilot, 1.9G scons cache
AVAILABLE_BYTES = 88 * GB - RESERVED_BYTES  # 88G partition
# AVAILABLE_BYTES = 100 * GB  # 100 GB

MIN_BYTES = 5 * GB
MIN_PERCENT = 10

def calculate_used_space(routes_training: list[str], routes_dashcam: list[str]) -> int:
  return len(routes_training) * TRAINING + len(routes_dashcam) * DASHCAM

def get_available_percent(routes_training: list[str], routes_dashcam: list[str]) -> float:
  return 100 * (1 - calculate_used_space(routes_training, routes_dashcam) / AVAILABLE_BYTES)

def get_available_bytes(routes_training: list[str], routes_dashcam: list[str]) -> int:
  return AVAILABLE_BYTES - calculate_used_space(routes_training, routes_dashcam)

get_available_percent([], [])

100.0

In [8]:
class Logger:
  route_counter = 0

  def __init__(self):
    self.route_counter = 0

  def __next__(self) -> str:
    log_id = f"{self.route_counter:010}"
    self.route_counter += 1
    return log_id

next(Logger())

'0000000000'

In [9]:
def deleter_original(routes_training: list[str], routes_dashcam: list[str]):
  while True:
    out_of_bytes = get_available_bytes(routes_training, routes_dashcam) < MIN_BYTES
    out_of_percent = get_available_percent(routes_training, routes_dashcam) < MIN_PERCENT
    if not (out_of_bytes or out_of_percent):
      return

    # TODO: avoid deleting most recent N preserved segments

    if len(routes_training) == 0 and len(routes_dashcam) == 0:
      return

    # remove earliest route from both lists
    routes_training.pop(0)
    routes_dashcam.pop(0)

In [10]:
MAX_TRAINING_BYTES = 5 * GB
MAX_TRAINING_MINUTES = MAX_TRAINING_BYTES / TRAINING

def deleter_new(routes_training: list[str], routes_dashcam: list[str]):
  while True:
    # If more than 5% of space is being used by training data, remove the earliest route from training data
    out_of_training_bytes = len(routes_training) > MAX_TRAINING_MINUTES
    if out_of_training_bytes:
      routes_training.pop(0)
      continue

    out_of_bytes = get_available_bytes(routes_training, routes_dashcam) < MIN_BYTES
    out_of_percent = get_available_percent(routes_training, routes_dashcam) < MIN_PERCENT
    if not (out_of_bytes or out_of_percent):
      return

    # TODO: avoid deleting most recent N preserved segments

    if len(routes_dashcam) == 0:
      return
    routes_dashcam.pop(0)

print(f"Preserving {MAX_TRAINING_MINUTES:.1f} minutes of training data (up to {MAX_TRAINING_BYTES/GB:.2f}GB)")

Preserving 18.6 minutes of training data (up to 5.00GB)


In [11]:
import pandas as pd


def loop(deleter, count: int):
  logger = Logger()
  routes_training = []
  routes_dashcam = []

  header = ["training_count", "dashcam_count", "training_bytes", "dashcam_bytes", "total_bytes", "available_bytes", "available_percent"]
  rows = []

  for _ in range(count):
    # 1. log new segment
    log_id = next(logger)
    routes_training.append(log_id)
    routes_dashcam.append(log_id)

    # 2. run deleter
    deleter(routes_training, routes_dashcam)

    # 3. append stats
    training_count, dashcam_count = len(routes_training), len(routes_dashcam)
    rows.append([
      training_count,
      dashcam_count,
      training_count * TRAINING,
      dashcam_count * DASHCAM,
      training_count * TRAINING + dashcam_count * DASHCAM,
      get_available_bytes(routes_training, routes_dashcam),
      get_available_percent(routes_training, routes_dashcam),
    ])

  return pd.DataFrame(rows, columns=header)

In [12]:
df = loop(deleter_original, 1024)
df

,training_count,dashcam_count,training_bytes,dashcam_bytes,total_bytes,available_bytes,available_percent
0,1,1,2.883584e+08,5.232394e+07,3.406823e+08,8.598816e+10,99.605367
1,2,2,5.767168e+08,1.046479e+08,6.813647e+08,8.564748e+10,99.210733
2,3,3,8.650752e+08,1.569718e+08,1.022047e+09,8.530680e+10,98.816100
3,4,4,1.153434e+09,2.092958e+08,1.362729e+09,8.496611e+10,98.421467
4,5,5,1.441792e+09,2.616197e+08,1.703412e+09,8.462543e+10,98.026834
...,...,...,...,...,...,...,...
1019,228,228,6.574572e+10,1.192986e+10,7.767557e+10,8.653269e+09,10.023612
1020,228,228,6.574572e+10,1.192986e+10,7.767557e+10,8.653269e+09,10.023612
1021,228,228,6.574572e+10,1.192986e+10,7.767557e+10,8.653269e+09,10.023612
1022,228,228,6.574572e+10,1.192986e+10,7.767557e+10,8.653269e+09,10.023612


In [13]:
df = loop(deleter_new, 9999)
df

,training_count,dashcam_count,training_bytes,dashcam_bytes,total_bytes,available_bytes,available_percent
0,1,1,2.883584e+08,5.232394e+07,3.406823e+08,8.598816e+10,99.605367
1,2,2,5.767168e+08,1.046479e+08,6.813647e+08,8.564748e+10,99.210733
2,3,3,8.650752e+08,1.569718e+08,1.022047e+09,8.530680e+10,98.816100
3,4,4,1.153434e+09,2.092958e+08,1.362729e+09,8.496611e+10,98.421467
4,5,5,1.441792e+09,2.616197e+08,1.703412e+09,8.462543e+10,98.026834
...,...,...,...,...,...,...,...
9994,18,1385,5.190451e+09,7.246866e+10,7.765911e+10,8.669731e+09,10.042682
9995,18,1385,5.190451e+09,7.246866e+10,7.765911e+10,8.669731e+09,10.042682
9996,18,1385,5.190451e+09,7.246866e+10,7.765911e+10,8.669731e+09,10.042682
9997,18,1385,5.190451e+09,7.246866e+10,7.765911e+10,8.669731e+09,10.042682
